In [ ]:
import os, shutil, zipfile
from glob import glob
from os.path import exists, join
from pprint import pprint as pp
import numpy as np
from bs4 import BeautifulSoup
from pydub import AudioSegment

import matplotlib.pyplot as plt
import wave
import time
import pyperclip ;# #pyperclip.copy('Theclipboard.'); pyperclip.paste()

from pydub import AudioSegment
from pydub.playback import play
from random import randint
import librosa as lbs

In [ ]:
#!pip install bs4
#!pip install matplotlib
#!pip install mkl-random
#!pip install mkl-fft
#!pip install ffmpeg-normalize
#!pip install librosa
#!pip install soundfile
#!pip install pyperclip ; #pyperclip.copy('Theclipboard.'); pyperclip.paste()
#sudo apt-get install python-tk

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
pPath      ='/home/u/2018/grgDBs/test'
pOut_Folder='/home/u/2018/grgDBs/test_out'

#fnname=join(pPath,"file10.wav") # "810920017105505.wav")
pWavsInput=glob(join(pPath,'*.wav'))
pWavs=glob(join(pOut_Folder,'*.wav'))
if not exists(pOut_Folder): os.makedirs(pOut_Folder)
pWavs=sorted(pWavs)
pp(pWavsInput)
pp(pWavs)


1. Mở file audio
    - Duyệt theo thư mục
2. Chia theo thời gian đã làm manual
3. Chia theo silent bằng thư viện pydub

In [ ]:
#for pWav in pWavs:
#    print(pWav)

from scipy.io import wavfile
fs, data = wavfile.read(fnname)

from IPython.display import Audio

Audio(url=fnname, autoplay=True)


from pydub import AudioSegment
from pydub.playback import play

song = AudioSegment.from_wav(fnname)
play(song)

plt.figure(num=None, figsize=(12, 3), dpi=80, facecolor='w', edgecolor='k')
plt.title('Signal Wave...')
plt.plot(data)
plt.show()

In [ ]:

def monoWavRead(filename=''):
    #print('=============trong ham doc file:',filename)
    x,fs = lbs.load(filename,res_type='kaiser_fast')
    if (x.ndim==1):
        samples = x
    else:
        samples = x[:, 0]
    return fs, samples
def write(filename, rate , data  ):
    #print('=============trong ham write file:',filename)
    lbs.output.write_wav(filename, y=data, sr=rate)
    
    #lbs.output.write_wav(filename, y=data.astype(np.int16), sr=rate)
    #print('write wav ok')
    
    #y=data.astype(np.int16): view
    # https://stackoverflow.com/questions/47870412/librosa-cant-open-wav-created-by-librosa
    
def FnNormalize(data0):
    print('FnNormalize -------------------')
    MA=max(data0)
    MI=min(data0)
    if MA<-MI:MA=-MI
    data=data0/MA
    return data

def FnXacDinhViTriCat(data,db=0.03):
    print('FnXacDinhViTriCat -------------------')
    Sile=[]
    for samp in data:
        if (samp> -db) and (samp<db):
            Sile.append(0)
        else:
            Sile.append(1)
    #### - Xac dinh vi tri can cat---------------------------        
    SS=[]
    alpha=0.99                 #~~~~~~~~~~~~~ Threshole ~~~~~~~~~~~~
    VL=0 
    for X in Sile:
        VL=VL*alpha+X*(1-alpha)
        SS.append(VL)

    SS2=[]    
    VL2=0 
    alpha2=0.998               #~~~~~~~~~~~~~ Threshole ~~~~~~~~~~~~
    for X in SS:
        VL2=VL2*alpha2+X*(1-alpha2)
        SS2.append(VL2)
    for k,X in enumerate(SS2):
        if X>db:
            SS[k]=1
        else:
            SS[k]=0
    return SS
def FnFixFirstPoint(SS2):
    print('FnFixFirstPoint---------------------------------')
    #--- Fix first points (=0 la sai, doan dau=1)-------
    SS3=[]
    beta=0.1
    count=0
    f1=0
    MaxN=1#18000
    for samp in SS2:
        if (samp<beta):
            count+=1
            if count>1500:
                SS3.append(0)   
            else:
                SS3.append(MaxN)   
        else:
            SS3.append(MaxN)
            count=0
    #--- shift zero areas ------------------------------
    Delta=2000
    for samp in range(0,len(SS3)-Delta-1):
        if((SS3[samp+Delta]>0)and(SS3[samp]==0))or(SS3[samp]>0):SS2[samp]=MaxN/2
        if((SS3[samp+Delta//2]==0)and(SS3[samp]>0)):            SS2[samp]=0
        #if(samp<10):print(samp, SS2[samp])

    for k,samp in enumerate(SS2):
        if(samp)<MaxN/4:
            SS2[k]=MaxN/4
    return SS2

def FnCutList(SS2,Silent_length=2000):
    print('FnCutList -------------------')
    #Cut list:------------------------------------------
    CUTs=[]
    d=0;c=0
    Dau=0
    MaxN=1
    print(max(SS2), min(SS2), len(SS2))
    for k in range(0,len(SS2)-1):        
        if SS2[k]>SS2[k+1]: d=k            #print(k,SS2[k])
        if SS2[k]<SS2[k+1]: c=k            #print(k,SS2[k])
        if (d>0) and (c>0):            #print('dau,cuoi=',d,c)
            if (c-d>=Silent_length)or(d-c>=Silent_length): ############ Adjust len of silent
                CUTs.append([Dau,d+500])
                Dau=c-500                #print(d,c)
            d=0;c=0     
    SS3=np.zeros(len(SS2))
    for k in range(0,len(SS3)):
        SS3[k]=-MaxN
    for [d,c] in CUTs:
        SS3[d]=-MaxN/10
        SS3[c]=-MaxN/10
    print('len(CUTs)=',len(CUTs))    
    return [CUTs,SS3]
def format_seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    if hours>0:
        return "%02i.%02i.%04.1f" % (hours, minutes, seconds)
    else:
        if minutes>0:
            return "%02i.%04.1f" % (minutes, seconds)
        else:
            return "%04.1f" % (seconds)

def FnPlot(dat):
    fig=plt.figure(num=None, figsize=(30,5), facecolor='w', edgecolor='k')
    plt.plot(dat)
    plt.show()
def FnPlotM(DATs):
    fig=plt.figure(num=None, figsize=(30, 5), facecolor='w', edgecolor='k')
    for dat in DATs:
        plt.plot(dat)
    plt.show()

In [ ]:
saveFigure=1 #### Set=1 to save that sement audio to png 
for fn in pWavsInput:
    print(fn)
    #fs, data = wavfile.read('/home/u/2018/grgDBs/test/810920017105520.wav' )#fnname)
    #fn='/home/u/2018/grgDBs/test/810920017105543.wav'
    #data = AudioSegment.from_file(fn)
    #data.export(fn, format="wav")
    #fs, data = wavfile.read('/home/u/2018/grgDBs/test/810920017105520.wav' )#fnname)
    # data rang: (-18743, 17930)
    #---------------------------------------------------
    fs, data0 = monoWavRead(fn) 
    #print(max(data0),min(data0),fs,len(data0))
    #data0=data0[:5000000]
    #print(max(data0),min(data0),fs,len(data0))

    data=FnNormalize(data0)
    #print(max(data),min(data))

    db=0.02         ############## Threshold=1000, phan biet tin hieu va nhieu
    SS2=FnXacDinhViTriCat(data,db) 
    #print('len(SS2)=',len(SS2))
    #FnPlotM([data,SS2])
    SS2= FnFixFirstPoint(SS2)
    #FnPlotM([data,SS2])
    CUTs,SS3 = FnCutList(SS2)
    #print('len(CUTs[{}],SS3[{}])'.format(len(CUTs),len(SS3)))

    #================================================================================================

    #print('FnCutList -------------------')
    Secments=len(CUTs)
    MaxDuaration=len(data)/fs
    #print('Secments=',Secments)
    for k in range(Secments):
        X=data[CUTs[k][0]:CUTs[k][1]]
        outfile=fn.split('/')[-1]
        duration0 = format_seconds_to_hhmmss(CUTs[k][0] / float(fs))
        duration1 = format_seconds_to_hhmmss(CUTs[k][1] / float(fs))
        if (Secments>99):
            sFn=    "_{0:03d}_{1:03d}_({2}-{3})".format(Secments,k,duration0,duration1)
        else:
            sFn=    "_{0:02d}_{1:02d}_({2}-{3})".format(Secments,k,duration0,duration1)
        outfile=outfile.replace('.wav',sFn+'.wav')
        #print(outfile)
        write(filename = join(pOut_Folder, outfile), rate = fs, data = X)
    outfile, fs, len(data),'second=',len(data)/fs
    if (saveFigure==1):
        fig=plt.figure(num=None, figsize=(30, 5), facecolor='w', edgecolor='k')
        plt.plot(X)
        outfile=outfile.replace('.wav','.png')
        fig.savefig(join(pOut_Folder, outfile), format='png', dpi=200) 

print('Done! Produce: Wav, png')

for kk in range(len(CUTs)):
    fig=plt.figure(num=None, figsize=(30, 5), facecolor='w', edgecolor='k')
    plt.title('Zero...')
    Fr=100000
    d=kk*Fr
    t=(kk+1)*Fr
    plt.plot(data[d:t])
    plt.plot(SS2[d:t])
    plt.plot(SS3[d:t])
    plt.show()
print(len(SS2))
fig.savefig(join(pOut_Folder,'myimage.svg'), format='svg', dpi=1200)#svg

In [ ]:
for k in range(len(CUTs)):
#if 1==1:
    print('k=',k)
    X=data[CUTs[k][0]:CUTs[k][1]]
    fig=plt.figure(num=None, figsize=(30, 5), facecolor='w', edgecolor='k')
    plt.title('Zero...')
    plt.plot(X)
    plt.show()
    #fig.savefig(join(ptest_out_slow80,'img{}.png'.format(k)), format='png', dpi=200)#svg

In [ ]:
import pyautogui  
#https://chrome.google.com/webstore/detail/keyboard-shortcuts-for-go/akjhnbnjanndggbcegmdggfjjclohjpo/related
pOut_Folder='/home/u/2018/grgDBs/test_out'
ptest_out_slow80='/home/u/2018/grgDBs/test_out_slow80'

if not exists(ptest_out_slow80): os.makedirs(ptest_out_slow80)
    
pWavs_out=glob(join(pOut_Folder,'*.wav'))
pWavs_out=sorted(pWavs_out)
 
len(pWavs_out),pWavs_out

In [ ]:
# Convert Audio files to Slowdown them: -------------------
for infile in pWavs_out:
    outfile=infile.replace(pOut_Folder,test_out_slow80) 
    #/home/u/2018/grgDBs/test_out_slow80
    S="ffmpeg -i '{}' -filter:a 'atempo=0.8' -vn '{}'".format(infile,outfile)
    #print(S)
    st=os.system(S)
    if (st!=0):
        print(st)
    


In [ ]:
# Get Position of mouse, run one time: -------------------
# Open translate.google.cn, half-left it, put mouse on that window area then run:
x, y = pyautogui.position()
positionStr = 'X: ' + str(x).rjust(4) + ' Y: ' + str(y).rjust(4)
print(positionStr, end='')
print('\b' * len(positionStr), end='', flush=True)
# half left size: X: 1772 Y:  41

In [ ]:
# Auto Speech recognition by tool:
# plug Audio line out to line in then run:
import timeit

test_out_slow80=glob(join(ptest_out_slow80,'*.wav'))
test_out_slow80=sorted(test_out_slow80)
Trans=[]
t=randint(3,5)
for k,fn in enumerate(test_out_slow80):
    pyautogui.click(x,y)
    time.sleep(t)
    pyautogui.hotkey('alt', 'm')
    #print('hotkey m.')
    start = timeit.default_timer()
    song = AudioSegment.from_wav(fn)
    play(song)
    stop = timeit.default_timer()
    tt=stop - start 
    time.sleep(t+tt/2)
    
    pyautogui.click(x,y)
    time.sleep(tt/2)
    #pyautogui.click(x-100,y+10)
    #time.sleep(tt/2)
    
    pyautogui.hotkey('alt', 'm')
    pyautogui.hotkey('ctrl', 'c')
    #pyautogui.hotkey('ctrl', 'a')
    #pyautogui.hotkey('ctrl', 'x')
    pyautogui.hotkey('alt', 'd')
    vl=pyperclip.paste()
    print(k,':',vl)
    Trans.append(vl)
    pyperclip.copy("")
    time.sleep(t)
    if k%10==0:
        pyautogui.hotkey('f5')
        time.sleep(3)

aa, bb = pyautogui.position()
positionStr = 'X: ' + str(aa).rjust(4) + ' Y: ' + str(bb).rjust(4)
print(positionStr, end='')
print('\b' * len(positionStr), end='', flush=True)


In [ ]:
# Save labels text to files:
for k, ff in enumerate(test_out_slow80):
    with open(ff.replace('.wav','.txt'),'w') as fout:
        fout.write(Trans[k])
    print(Trans[k])

In [ ]:
#Create HTML file to test result:

ss='<hr/><br/>	\
<button onclick="{7}()" type="button">{0}</button>\
<text id="{8}"></text> \
<div id="{0}"> \
	<audio id="{1}"><source src="{2}" type="audio/mpeg"></audio> \
	<button onclick="{3}()" type="button">Play Audio</button> \
	<input id="{4}" type="text" size="80" onkeydown="{5}(event)" value="{6}"> \
	<script> \
		var {10} = document.getElementById("{1}");  \
		function {3}() @ {10}.play(); $  \
		function {9}()@ {10}.pause();$ \
		function {7}()@ \
			document.getElementById("{8}").innerHTML = "OK!";\
            var vv=document.getElementById("{0}").style.display;\
            if (vv!="none")@document.getElementById("{0}").style.display = "none";$\
            else @document.getElementById("{0}").style.display = "inline";$\
			$\
		function {5}(event) @\
			var x =  event.keyCode; \
			if (x == 27) @{3}();$\
			if (x == 17) @{9}();$\
		$\
	</script>\
</div>'

ss1=''
for k,ff in enumerate(test_out_slow80):
    ss1+=ss.format( 'Audio %s'%k,
                  'au%s'%k,
                  ff.split('/')[-1],
                  'pAu%s'%k,
                  'tact%s'%k,
                  'PLAY%s'%k,
                  Trans[k],
                  'save%s'%k,
                  'ok%s'%k,
                  'pausA%s'%k,
                  'x%s'%k)
ss1=ss1.replace('@','{')
ss1=ss1.replace('$','}')
ssfile='<!DOCTYPE html>\
<html>\
<body>\
<p>1. Click on text <br/>\
2. Press ESC to play audio<br/>\
3. Press CTR to pause</p>'

ssfileend='</body></html>'
with open(join(ptest_out_slow80,'00.html'),'w') as ff:
    ff.write(ssfile)
    ff.write(ss1)
    ff.write(ssfileend)     
print('View result in:',join(ptest_out_slow80,'00.html'))    